## Whitehat Hack Funds Disbursement
We need to disburse funds to users whose funds we rescued. The purpose of this notebook is to have a single source of truth for how many funds we owe each user.

#### Decisions / Assumptions
* The point in time we will use to calculate a users assets in the protocol will be the block before the first whitehat on each chain


In [12]:
import os
from web3 import Web3
from dotenv import load_dotenv
import requests
import json
from decimal import Decimal, ROUND_DOWN
import csv    
import pandas as pd
import time
import random

In [13]:
"""
   _   _   _   _   _   _   _   _   _  
  / \ / \ / \ / \ / \ / \ / \ / \ / \ 
 ( C | o | n | s | t | a | n | t | s )
  \_/ \_/ \_/ \_/ \_/ \_/ \_/ \_/ \_/ 
"""

####################################################
# Environment variables
####################################################
load_dotenv()
ALCHEMY_API_KEY = os.getenv('ALCHEMY_API_KEY')

####################################################
# RPC Clients
####################################################
w3_mainnet = Web3(Web3.HTTPProvider(f'https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))
w3_base = Web3(Web3.HTTPProvider(f'https://base-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))
w3_unichain = Web3(Web3.HTTPProvider(f'https://unichain-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}'))

chain_to_rpc_client = {
    1: w3_mainnet,
    8453: w3_base,
    130: w3_unichain
}

####################################################
# Whitehat block nums
####################################################
panoptic_chains = [130, 8453, 1]

chain_to_whitehat_blocks = {
    # first whitehat txn block mainnet, and closest blocks by timestamp on base and unichain
    1: 23242436,    # mainnet
    8453: 34814389, # base
    130:  25669765  # unichain

    # blocks closest to timestamp for mainnet block
    # first whitehat txn blocks
    # 130: 25669817,  # unichain
    # 8453: 34814419, # base
}

####################################################
# Chain id -> chain name
####################################################
chain_id_to_chain_name = {
  1: 'ethereum',
  130: 'unichain',
  8453: 'base'
}

####################################################
# Subgraph urls
####################################################
chain_to_subgraph_url = {
    # mainnet
    1: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-mainnet/dev/gn',
    # unichain
    130: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-unichain/dev/gn',
    # base
    8453: 'https://api.goldsky.com/api/public/project_cl9gc21q105380hxuh8ks53k3/subgraphs/panoptic-subgraph-base/dev/gn',
}

####################################################
# Contract addrs
####################################################

chain_to_contract_addrs = {
    # mainnet
    1: {
        'SemiFungiblePositionManager': '0x0000000000000DEdEDdD16227aA3D836C5753194',
        'PanopticFactory': '0x000000000000010a1DEc6c46371A28A071F8bb01',
        'PanopticPoolTemplate': '0x0000000000001B1A7fe31692d107cAA42fb06862',
        'UniswapV3Factory': '0x1F98431c8aD98523631AE4a59f267346ea31F984',
        'NonFungiblePositionManager': '0xc36442b4a4522e871399cd717abdd847ab11fe88',
        'PanopticQuery': '0x95A5fF032C728F7F16e7fBd4F4EA2d0C0623Ffbb',
        'UniswapHelper': '0x2049aa04779e56aB54585dA0c3639f139a37A168',
        'UniswapMigrator': '0xd6f8c13ec833ed8afabcd111adecc90a765b6ed9',
        'PoolManager': '0xE03A1074c86CFeDd5C142C4F04F1a1536e203543',
        'PanopticFactoryV1_1': '0x0000000000000CF008e9bf9D01f8306029724c80',
        'SemiFungiblePositionManagerV1_1': '0x0000000000000aAbbcfCA8100a9ee78124E97B33',
        'PanopticPoolTemplateV1_1': '0x0000000000035D9945Bf4d24393828e920376bAe',
        'StateView': '0x7ffe42c4a5deea5b0fec41c94c136cf115597227',
        'UniswapHelperV1_1': '0xfc6e5122a214d06bcbecf1d0db1130b4f1f846b8',
        'PanopticQueryV1_1': '0x518dB79DCFbb300109E3Aaf3dD79f046581ABd81',
        'PanopticMath': '0x000000000001CD07e625A9e225C37BEA50b3F441',
        'PanopticMathV1_1': '0x000000000001CD07e625A9e225C37BEA50b3F441',
    },
    # unichain
    130: {
        'SemiFungiblePositionManager': '0x0000000000000DEdEDdD16227aA3D836C5753194',
        'PanopticFactory': '0x000000000000010a1DEc6c46371A28A071F8bb01',
        'PanopticPoolTemplate': '0x0000000000001B1A7fe31692d107cAA42fb06862',
        'UniswapV3Factory': '0x1F98400000000000000000000000000000000003',
        'NonFungiblePositionManager': '0x943e6e07a7E8E791dAFC44083e54041D743C46E9',
        'PanopticQuery': '0x9574ed459807e6b4f841da754164d6f6cb331b1d',
        'UniswapHelper': '0x0A7d8F96eD5d78E2B1c6211c7Ec81Ee7fD44d217',
        'UniswapMigrator': '0x0000000000000000000000000000000000000000',
        'PoolManager': '0xE03A1074c86CFeDd5C142C4F04F1a1536e203543',
        'PanopticFactoryV1_1': '0x0000000000000CF008e9bf9D01f8306029724c80',
        'SemiFungiblePositionManagerV1_1': '0x0000000000000aAbbcfCA8100a9ee78124E97B33',
        'PanopticPoolTemplateV1_1': '0x0000000000035D9945Bf4d24393828e920376bAe',
        'StateView': '0x86e8631a016f9068c3f085faf484ee3f5fdee8f2',
        'UniswapHelperV1_1': '0xdc3b61181b041985f570237b9edc237d35789b97',
        'PanopticQueryV1_1': '0x70d973b11ae0937c4e29981cdc8ca4afe2e27959',
        'PanopticMath': '0x000000000001CD07e625A9e225C37BEA50b3F441',
        'PanopticMathV1_1': '0x000000000001CD07e625A9e225C37BEA50b3F441',
    },
    # base
    8453: {
        'SemiFungiblePositionManager': '0x0000000000000DEdEDdD16227aA3D836C5753194',
        'PanopticFactory': '0x000000000000010a1DEc6c46371A28A071F8bb01',
        'PanopticPoolTemplate': '0x0000000000001B1A7fe31692d107cAA42fb06862',
        'UniswapV3Factory': '0x33128a8fC17869897dcE68Ed026d694621f6FDfD',
        'NonFungiblePositionManager': '0x03a520b32c04bf3beef7beb72e919cf822ed34f1',
        'PanopticQuery': '0x0b9b661affb5548ec32e36a0730f7ad579d18ac4',
        'UniswapHelper': '0xf28143e3356dbcaf36b52b2a5fff976fb22e55ef',
        'UniswapMigrator': '0x0000000000000000000000000000000000000000',
        'PoolManager': '0x498581ff718922c3f8e6a244956af099b2652b2b',
        'PanopticFactoryV1_1': '0x0000000000000CF008e9bf9D01f8306029724c80',
        'SemiFungiblePositionManagerV1_1': '0x0000000000000aAbbcfCA8100a9ee78124E97B33',
        'PanopticPoolTemplateV1_1': '0x0000000000035D9945Bf4d24393828e920376bAe',
        'StateView': '0xa3c0c9b65bad0b08107aa264b0f3db444b867a71',
        'UniswapHelperV1_1': '0xea59473cdd44b859bef7270a5aa8b8758c88f8b1',
        'PanopticQueryV1_1': '0x5aa79223cd973d341d29b41d803f23a88816d904',
        'PanopticMath': '0x000000000001CD07e625A9e225C37BEA50b3F441',
        'PanopticMathV1_1': '0x000000000001CD07e625A9e225C37BEA50b3F441',
    }
}

####################################################
# Token addrs
####################################################
mainnet_weth = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
mainnet_wbtc = '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'
mainnet_usdc = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
mainnet_tbtc = '0x18084fbA666a33d37592fA2633fD49a74DD93a88'

base_weth = '0x4200000000000000000000000000000000000006'
base_usdc = '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913'
base_cbBTC = '0xcbB7C0000aB88B473b1f5aFd9ef808440eed33Bf'

unichain_usdt = '0x9151434b16b9763660705744891fA906F660EcC5'
unichain_usdc = '0x078D782b760474a361dDA0AF3839290b0EF57AD6'
unichain_wbtc = '0x927B51f251480a681271180DA4de28D44EC4AfB8'
unichain_weth = '0x4200000000000000000000000000000000000006'

####################################################
# Whitehat metadata
####################################################
attacker = '0xa1F0A9d51b592ee074eD6987006976908631503B'

panoptic_safe_address = '0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1'

####################################################
# Abis
####################################################
# Minimal ABI for balanceOf
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function"
    }
]

# minimal abi for twapFilter
PANOPTIC_MATH_ABI = [
    {
        "type": "function",
        "name": "twapFilter",
        "inputs": [
                {
                    "name": "univ3pool",
                    "type": "address",
                    "internalType": "address"
                },
            {"name": "twapWindow", "type": "uint32", "internalType": "uint32"}
        ],
        "outputs": [{"name": "", "type": "int24", "internalType": "int24"}],
        "stateMutability": "view"
    }
]

PANOPTIC_QUERY_ABI = [
    {
        "inputs": [
            {
                "internalType": "address",
                "name": "pool",
                "type": "address"
            },
            {
                "internalType": "address", 
                "name": "account",
                "type": "address"
            },
            {
                "internalType": "bool",
                "name": "includePendingPremium",
                "type": "bool"
            },
            {
                "internalType": "uint256[]",
                "name": "positionIdList",
                "type": "uint256[]"
            },
            {
                "internalType": "int24",
                "name": "atTick",
                "type": "int24"
            }
        ],
        "name": "getNetLiquidationValue",
        "outputs": [
            {
                "internalType": "int256",
                "name": "value0",
                "type": "int256"
            },
            {
                "internalType": "int256", 
                "name": "value1",
                "type": "int256"
            }
        ],
        "stateMutability": "view",
        "type": "function"
    }
]

PANOPTIC_POOL_ABI = [
    {
      "type": "constructor",
      "inputs": [
        {
          "name": "_sfpm",
          "type": "address",
          "internalType": "contract SemiFungiblePositionManager"
        },
        { "name": "_poolManager", "type": "address", "internalType": "contract IPoolManager" }
      ],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "assertMinCollateralValues",
      "inputs": [
        { "name": "minValue0", "type": "uint256", "internalType": "uint256" },
        { "name": "minValue1", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "burnOptions",
      "inputs": [
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "newPositionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "tickLimitLow", "type": "int24", "internalType": "int24" },
        { "name": "tickLimitHigh", "type": "int24", "internalType": "int24" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "burnOptions",
      "inputs": [
        { "name": "tokenId", "type": "uint256", "internalType": "TokenId" },
        { "name": "newPositionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "tickLimitLow", "type": "int24", "internalType": "int24" },
        { "name": "tickLimitHigh", "type": "int24", "internalType": "int24" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "collateralToken0",
      "inputs": [],
      "outputs": [{ "name": "", "type": "address", "internalType": "contract CollateralTracker" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "collateralToken1",
      "inputs": [],
      "outputs": [{ "name": "", "type": "address", "internalType": "contract CollateralTracker" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "forceExercise",
      "inputs": [
        { "name": "account", "type": "address", "internalType": "address" },
        { "name": "tokenId", "type": "uint256", "internalType": "TokenId" },
        { "name": "positionIdListExercisee", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "positionIdListExercisor", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "usePremiaAsCollateral", "type": "uint256", "internalType": "LeftRightUnsigned" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "getAccumulatedFeesAndPositionsData",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "includePendingPremium", "type": "bool", "internalType": "bool" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" }
      ],
      "outputs": [
        { "name": "", "type": "uint256", "internalType": "LeftRightUnsigned" },
        { "name": "", "type": "uint256", "internalType": "LeftRightUnsigned" },
        { "name": "", "type": "uint256[2][]", "internalType": "uint256[2][]" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "getOracleTicks",
      "inputs": [],
      "outputs": [
        { "name": "currentTick", "type": "int24", "internalType": "int24" },
        { "name": "fastOracleTick", "type": "int24", "internalType": "int24" },
        { "name": "slowOracleTick", "type": "int24", "internalType": "int24" },
        { "name": "latestObservation", "type": "int24", "internalType": "int24" },
        { "name": "medianData", "type": "uint256", "internalType": "uint256" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "initialize",
      "inputs": [],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "isSafeMode",
      "inputs": [],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "liquidate",
      "inputs": [
        { "name": "positionIdListLiquidator", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "liquidatee", "type": "address", "internalType": "address" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" }
      ],
      "outputs": [],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "mintOptions",
      "inputs": [
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "positionSize", "type": "uint128", "internalType": "uint128" },
        { "name": "effectiveLiquidityLimitX32", "type": "uint64", "internalType": "uint64" },
        { "name": "tickLimitLow", "type": "int24", "internalType": "int24" },
        { "name": "tickLimitHigh", "type": "int24", "internalType": "int24" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "multicall",
      "inputs": [{ "name": "data", "type": "bytes[]", "internalType": "bytes[]" }],
      "outputs": [{ "name": "results", "type": "bytes[]", "internalType": "bytes[]" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "numberOfLegs",
      "inputs": [{ "name": "user", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "onERC1155Received",
      "inputs": [
        { "name": "", "type": "address", "internalType": "address" },
        { "name": "", "type": "address", "internalType": "address" },
        { "name": "", "type": "uint256", "internalType": "uint256" },
        { "name": "", "type": "uint256", "internalType": "uint256" },
        { "name": "", "type": "bytes", "internalType": "bytes" }
      ],
      "outputs": [{ "name": "", "type": "bytes4", "internalType": "bytes4" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "oracleContract",
      "inputs": [],
      "outputs": [
        { "name": "", "type": "address", "internalType": "contract IV3CompatibleOracle" }
      ],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "pokeMedian",
      "inputs": [],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "poolKey",
      "inputs": [],
      "outputs": [
        {
          "name": "key",
          "type": "tuple",
          "internalType": "struct PoolKey",
          "components": [
            { "name": "currency0", "type": "address", "internalType": "Currency" },
            { "name": "currency1", "type": "address", "internalType": "Currency" },
            { "name": "fee", "type": "uint24", "internalType": "uint24" },
            { "name": "tickSpacing", "type": "int24", "internalType": "int24" },
            { "name": "hooks", "type": "address", "internalType": "contract IHooks" }
          ]
        }
      ],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "positionData",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "tokenId", "type": "uint256", "internalType": "TokenId" }
      ],
      "outputs": [
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int24", "internalType": "int24" },
        { "name": "", "type": "int256", "internalType": "int256" },
        { "name": "", "type": "int256", "internalType": "int256" },
        { "name": "", "type": "uint128", "internalType": "uint128" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "settleLongPremium",
      "inputs": [
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "owner", "type": "address", "internalType": "address" },
        { "name": "legIndex", "type": "uint256", "internalType": "uint256" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "validateCollateralWithdrawable",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [],
      "stateMutability": "view"
    },
    { "type": "error", "name": "AccountInsolvent", "inputs": [] },
    { "type": "error", "name": "CastingError", "inputs": [] },
    { "type": "error", "name": "EffectiveLiquidityAboveThreshold", "inputs": [] },
    { "type": "error", "name": "InputListFail", "inputs": [] },
    { "type": "error", "name": "InvalidTick", "inputs": [] },
    {
      "type": "error",
      "name": "InvalidTokenIdParameter",
      "inputs": [{ "name": "parameterType", "type": "uint256", "internalType": "uint256" }]
    },
    { "type": "error", "name": "NoLegsExercisable", "inputs": [] },
    { "type": "error", "name": "NotALongLeg", "inputs": [] },
    { "type": "error", "name": "NotMarginCalled", "inputs": [] },
    { "type": "error", "name": "PoolAlreadyInitialized", "inputs": [] },
    { "type": "error", "name": "PositionAlreadyMinted", "inputs": [] },
    { "type": "error", "name": "StaleOracle", "inputs": [] },
    { "type": "error", "name": "TooManyLegsOpen", "inputs": [] },
    { "type": "error", "name": "UnderOverFlow", "inputs": [] }
  ]

COLLATERAL_TRACKER_ABI = [
    {
      "type": "constructor",
      "inputs": [
        { "name": "_commissionFee", "type": "uint256", "internalType": "uint256" },
        { "name": "_sellerCollateralRatio", "type": "uint256", "internalType": "uint256" },
        { "name": "_buyerCollateralRatio", "type": "uint256", "internalType": "uint256" },
        { "name": "_forceExerciseCost", "type": "int256", "internalType": "int256" },
        { "name": "_targetPoolUtilization", "type": "uint256", "internalType": "uint256" },
        { "name": "_saturatedPoolUtilization", "type": "uint256", "internalType": "uint256" },
        { "name": "_ITMSpreadFee", "type": "uint256", "internalType": "uint256" },
        { "name": "_manager", "type": "address", "internalType": "contract IPoolManager" }
      ],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "allowance",
      "inputs": [
        { "name": "owner", "type": "address", "internalType": "address" },
        { "name": "spender", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "allowance", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "approve",
      "inputs": [
        { "name": "spender", "type": "address", "internalType": "address" },
        { "name": "amount", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "asset",
      "inputs": [],
      "outputs": [{ "name": "assetTokenAddress", "type": "address", "internalType": "address" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "balanceOf",
      "inputs": [{ "name": "account", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "balance", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "convertToAssets",
      "inputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "convertToShares",
      "inputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "decimals",
      "inputs": [],
      "outputs": [{ "name": "", "type": "uint8", "internalType": "uint8" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "delegate",
      "inputs": [{ "name": "delegatee", "type": "address", "internalType": "address" }],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "deposit",
      "inputs": [
        { "name": "assets", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "exercise",
      "inputs": [
        { "name": "optionOwner", "type": "address", "internalType": "address" },
        { "name": "longAmount", "type": "int128", "internalType": "int128" },
        { "name": "shortAmount", "type": "int128", "internalType": "int128" },
        { "name": "swappedAmount", "type": "int128", "internalType": "int128" },
        { "name": "realizedPremium", "type": "int128", "internalType": "int128" }
      ],
      "outputs": [{ "name": "", "type": "int128", "internalType": "int128" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "exerciseCost",
      "inputs": [
        { "name": "currentTick", "type": "int24", "internalType": "int24" },
        { "name": "oracleTick", "type": "int24", "internalType": "int24" },
        { "name": "positionId", "type": "uint256", "internalType": "TokenId" },
        { "name": "positionSize", "type": "uint128", "internalType": "uint128" },
        { "name": "longAmounts", "type": "int256", "internalType": "LeftRightSigned" }
      ],
      "outputs": [{ "name": "exerciseFees", "type": "int256", "internalType": "LeftRightSigned" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "getAccountMarginDetails",
      "inputs": [
        { "name": "user", "type": "address", "internalType": "address" },
        { "name": "atTick", "type": "int24", "internalType": "int24" },
        { "name": "positionBalanceArray", "type": "uint256[2][]", "internalType": "uint256[2][]" },
        { "name": "shortPremium", "type": "uint128", "internalType": "uint128" },
        { "name": "longPremium", "type": "uint128", "internalType": "uint128" }
      ],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "LeftRightUnsigned" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "getPoolData",
      "inputs": [],
      "outputs": [
        { "name": "poolAssets", "type": "uint256", "internalType": "uint256" },
        { "name": "insideAMM", "type": "uint256", "internalType": "uint256" },
        { "name": "currentPoolUtilization", "type": "uint256", "internalType": "uint256" }
      ],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "initialize",
      "inputs": [],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "maxDeposit",
      "inputs": [{ "name": "", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxAssets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "pure"
    },
    {
      "type": "function",
      "name": "maxMint",
      "inputs": [{ "name": "", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxShares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "maxRedeem",
      "inputs": [{ "name": "owner", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxShares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "maxWithdraw",
      "inputs": [{ "name": "owner", "type": "address", "internalType": "address" }],
      "outputs": [{ "name": "maxAssets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "mint",
      "inputs": [
        { "name": "shares", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "multicall",
      "inputs": [{ "name": "data", "type": "bytes[]", "internalType": "bytes[]" }],
      "outputs": [{ "name": "results", "type": "bytes[]", "internalType": "bytes[]" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "name",
      "inputs": [],
      "outputs": [{ "name": "", "type": "string", "internalType": "string" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewDeposit",
      "inputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewMint",
      "inputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewRedeem",
      "inputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "previewWithdraw",
      "inputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "redeem",
      "inputs": [
        { "name": "shares", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" },
        { "name": "owner", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "assets", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "refund",
      "inputs": [
        { "name": "refunder", "type": "address", "internalType": "address" },
        { "name": "refundee", "type": "address", "internalType": "address" },
        { "name": "assets", "type": "int256", "internalType": "int256" }
      ],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "revoke",
      "inputs": [{ "name": "delegatee", "type": "address", "internalType": "address" }],
      "outputs": [],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "settleLiquidation",
      "inputs": [
        { "name": "liquidator", "type": "address", "internalType": "address" },
        { "name": "liquidatee", "type": "address", "internalType": "address" },
        { "name": "bonus", "type": "int256", "internalType": "int256" }
      ],
      "outputs": [],
      "stateMutability": "payable"
    },
    {
      "type": "function",
      "name": "symbol",
      "inputs": [],
      "outputs": [{ "name": "", "type": "string", "internalType": "string" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "takeCommissionAddData",
      "inputs": [
        { "name": "optionOwner", "type": "address", "internalType": "address" },
        { "name": "longAmount", "type": "int128", "internalType": "int128" },
        { "name": "shortAmount", "type": "int128", "internalType": "int128" },
        { "name": "swappedAmount", "type": "int128", "internalType": "int128" },
        { "name": "isCovered", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [
        { "name": "", "type": "uint32", "internalType": "uint32" },
        { "name": "", "type": "uint128", "internalType": "uint128" }
      ],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "totalAssets",
      "inputs": [],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "totalSupply",
      "inputs": [],
      "outputs": [{ "name": "", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "view"
    },
    {
      "type": "function",
      "name": "transfer",
      "inputs": [
        { "name": "recipient", "type": "address", "internalType": "address" },
        { "name": "amount", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "transferFrom",
      "inputs": [
        { "name": "from", "type": "address", "internalType": "address" },
        { "name": "to", "type": "address", "internalType": "address" },
        { "name": "amount", "type": "uint256", "internalType": "uint256" }
      ],
      "outputs": [{ "name": "", "type": "bool", "internalType": "bool" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "unlockCallback",
      "inputs": [{ "name": "data", "type": "bytes", "internalType": "bytes" }],
      "outputs": [{ "name": "", "type": "bytes", "internalType": "bytes" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "withdraw",
      "inputs": [
        { "name": "assets", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" },
        { "name": "owner", "type": "address", "internalType": "address" }
      ],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "nonpayable"
    },
    {
      "type": "function",
      "name": "withdraw",
      "inputs": [
        { "name": "assets", "type": "uint256", "internalType": "uint256" },
        { "name": "receiver", "type": "address", "internalType": "address" },
        { "name": "owner", "type": "address", "internalType": "address" },
        { "name": "positionIdList", "type": "uint256[]", "internalType": "TokenId[]" },
        { "name": "usePremiaAsCollateral", "type": "bool", "internalType": "bool" }
      ],
      "outputs": [{ "name": "shares", "type": "uint256", "internalType": "uint256" }],
      "stateMutability": "nonpayable"
    },
    { "type": "error", "name": "CastingError", "inputs": [] },
    { "type": "error", "name": "CollateralTokenAlreadyInitialized", "inputs": [] },
    { "type": "error", "name": "DepositTooLarge", "inputs": [] },
    { "type": "error", "name": "ExceedsMaximumRedemption", "inputs": [] },
    { "type": "error", "name": "InvalidTick", "inputs": [] },
    { "type": "error", "name": "NotPanopticPool", "inputs": [] },
    { "type": "error", "name": "PositionCountNotZero", "inputs": [] },
    { "type": "error", "name": "TransferFailed", "inputs": [] },
    { "type": "error", "name": "UnauthorizedUniswapCallback", "inputs": [] },
    { "type": "error", "name": "UnderOverFlow", "inputs": [] }
  ]

## Funds rescued on each chain and stored in Safe multisig

In [14]:
chain_to_safe_tokens = {}

#### Funds available to disburse from Mainnet safe (as of Sep 2)

In [15]:
# Can confirm funds match safe mainnet balances here:
# https://app.safe.global/home?safe=eth:0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1
mainnet_tue_sep_1_2025_block = 23275660

# Dictionary mapping token addresses to symbols for mainnet
mainnet_tokens = {
    mainnet_weth: 'WETH',
    mainnet_wbtc: 'WBTC', 
    mainnet_tbtc: 'tBTC',
    mainnet_usdc: 'USDC'
}

# Get balances for each token
balances = {}
for token_addr, symbol in mainnet_tokens.items():
    contract = w3_mainnet.eth.contract(address=token_addr, abi=ERC20_ABI)
    balance = contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=mainnet_tue_sep_1_2025_block)
    balances[token_addr] = balance

# Store in chain_to_safe_tokens
chain_to_safe_tokens[1] = balances

print("Mainnet funds to distribute")
for token_addr, balance in balances.items():
    print(f"{mainnet_tokens[token_addr]}: {balance}")

# Write to CSV
with open('ethereum_funds.csv', 'w') as f:
    f.write('chain_id,token_address,token_symbol,balance\n')
    for token_addr, balance in balances.items():
        f.write(f'1,{token_addr.lower()},{mainnet_tokens[token_addr]},{balance}\n')

Mainnet funds to distribute
WETH: 79973292639517791570
WBTC: 16056508
tBTC: 49858198424970731
USDC: 39578598443


#### Funds available to disburse from Base safe (as of Sep 2)

In [16]:
# Can confirm funds match safe base balances here:
# https://app.safe.global/balances?safe=base:0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1
base_tue_sep_1_2025_block = 35014837

# Dictionary mapping token addresses to symbols for base
base_tokens = {
    base_weth: 'WETH',
    base_usdc: 'USDC',
    base_cbBTC: 'cbBTC'
}

# Get balances for each token
balances = {}
for token_addr, symbol in base_tokens.items():
    contract = w3_base.eth.contract(address=token_addr, abi=ERC20_ABI)
    balance = contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=base_tue_sep_1_2025_block)
    balances[token_addr] = balance

# Store in chain_to_safe_tokens
chain_to_safe_tokens[8453] = balances

print("Base funds to distribute")
for token_addr, balance in balances.items():
    print(f"{base_tokens[token_addr]}: {balance}")

# Write to CSV
with open('base_funds.csv', 'w') as f:
    f.write('chain_id,token_address,token_symbol,balance\n')
    for token_addr, balance in balances.items():
        f.write(f'8453,{token_addr.lower()},{base_tokens[token_addr]},{balance}\n')

Base funds to distribute
WETH: 16970998345489694683
USDC: 16830578554
cbBTC: 2373372


#### Funds available to disburse from Unichain safe (as of Sep 2)

In [17]:
# Can confirm funds match safe unichain balances here:
# https://app.safe.global/balances?safe=unichain:0x82BF455e9ebd6a541EF10b683dE1edCaf05cE7A1
unichain_tue_sep_1_2025_block = 26070662

# Dictionary mapping token addresses to symbols for unichain
unichain_tokens = {
    unichain_weth: 'WETH',
    unichain_wbtc: 'WBTC',
    unichain_usdc: 'USDC',
    unichain_usdt: 'USDT'
}

# Get balances for each token
balances = {}
for token_addr, symbol in unichain_tokens.items():
    contract = w3_unichain.eth.contract(address=token_addr, abi=ERC20_ABI)
    balance = contract.functions.balanceOf(panoptic_safe_address).call(block_identifier=unichain_tue_sep_1_2025_block)
    balances[token_addr] = balance

# Store in chain_to_safe_tokens 
chain_to_safe_tokens[1] = balances

print("Unichain funds to distribute")
for token_addr, balance in balances.items():
    print(f"{unichain_tokens[token_addr]}: {balance}")

# Write to CSV
with open('unichain_funds.csv', 'w') as f:
    f.write('chain_id,token_address,token_symbol,balance\n')
    for token_addr, balance in balances.items():
        f.write(f'1,{token_addr.lower()},{unichain_tokens[token_addr]},{balance}\n')

Unichain funds to distribute
WETH: 5267119200645429045
WBTC: 1488291
USDC: 8975993158
USDT: 12979404


#### User fund snapshot methodology

> Block `t` is 1 block before the block where the first whitehat transaction on mainnet executed

To get the snapshot of funds each user had in the protocol at the time of the rescue, we calculate the following at block `t`, for each account with open positions or deposits in a Panoptic pool:
1. The net value of the account's options portfolio in token0 and token1, calculated by [`getNetLiquidationValue`](https://github.com/panoptic-labs/panoptic-v1-helper/blob/main/src/PanopticQuery.sol#L849) with arguments as follows:
    * pool - self explanatory
    * account - self explanatory
    * `includePendingPremium` - true
    * `positionIdList` - using open positions from subgraph at block `t`
    * `atTick` - using panoptic pool's TWAP tick at block `t`.
2. Calculate the share price at block `t` for collateral0 and collateral1 for each Panoptic pool (e.g. `share_price0 = `[`collateral0.totalAssets`](https://github.com/panoptic-labs/panoptic-v1-core/blob/df4dc38dee4fe29fd889cffaa8097dccc561e572/contracts/CollateralTracker.sol#L347-L351) `/` `collateral0.totalSupply`).
3. Fetch the user's collateral0 and collateral1 share balances at the block.
4. Convert the user's collateral balances to assets using the share price (`shares * share_price`).
5. Net resulting token0 and token1 assets to get each user's total funds at the time of the rescue.

For Ethereum mainnet, block `t` is 23242435, because the first whitehat transaction occured on block 23242436.

For Base, block `t` is 34814389, the Base block closest in time to Ethereum mainnet block 23242435

For Unichain, block `t` is 25669765, the Unichain block closest in time to Ethereum mainnet block 23242435

In [18]:
def get_share_prices_per_panoptic_pool(panoptic_pool_accounts, block, chain_id):
    rpc_client = chain_to_rpc_client[chain_id]
    share_prices_per_panoptic_pool = {}
    for ppa in panoptic_pool_accounts:
        # Add jitter to requests
        time.sleep(random.uniform(0, 0.3))

        panoptic_pool_id = ppa['panopticPool']['id']
        if panoptic_pool_id not in share_prices_per_panoptic_pool:
            try:
                collateral0_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral0']['id']), abi=COLLATERAL_TRACKER_ABI)
                total_assets_0 = Decimal(collateral0_contract.functions.totalAssets().call(block_identifier=block))
                total_supply_0 = Decimal(collateral0_contract.functions.totalSupply().call(block_identifier=block))

                collateral1_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral1']['id']), abi=COLLATERAL_TRACKER_ABI)
                total_assets_1 = Decimal(collateral1_contract.functions.totalAssets().call(block_identifier=block))
                total_supply_1 = Decimal(collateral1_contract.functions.totalSupply().call(block_identifier=block))
            except Exception as e:
                print(f"Error getting share prices: {e}")
                print(f"collateral0 address: {ppa['collateral0']['id']}")
                print(f"collateral1 address: {ppa['collateral1']['id']}")
                print(f"chain_id: {chain_id}")
                print(f"block: {block}")
                raise

            share_prices_per_panoptic_pool[panoptic_pool_id] = [
                total_assets_0 / total_supply_0,
                total_assets_1 / total_supply_1
            ]
    return share_prices_per_panoptic_pool


def get_oracle_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id):
    rpc_client = chain_to_rpc_client[chain_id]
    oracle_ticks_per_panoptic_pool = {}

    for ppa in panoptic_pool_accounts:
        # Add jitter to requests
        time.sleep(random.uniform(0, 0.3))

        panoptic_pool_id = ppa['panopticPool']['id']
        if panoptic_pool_id not in oracle_ticks_per_panoptic_pool:
            panoptic_pool_address = Web3.to_checksum_address(panoptic_pool_id)
            panoptic_pool_contract = rpc_client.eth.contract(address=panoptic_pool_address, abi=PANOPTIC_POOL_ABI)

            if panoptic_pool_id == '0x000002daae4a265e7e0af0917cf8efcf0376ecdf':
                # Since getOracleTicks() is reverting for this mainnet pool (PRIME/USDC), use the current tick at the mainnet whitehat block - 1 instead
                # repeated 5 times to mimic the interface of getOracleTicks()
                oracle_ticks_per_panoptic_pool[panoptic_pool_id] = [248277, 248277, 248277, 248277, 248277]
            else:
                try:
                    oracle_ticks_per_panoptic_pool[panoptic_pool_id] = panoptic_pool_contract.functions.getOracleTicks().call(block_identifier=block)
                except Exception as e:
                    print(f"Error getting oracle ticks: {e}")
                    print(f"oracle_ticks: {oracle_ticks_per_panoptic_pool}")
                    print(f"block: {block}")
                    print(f"chain_id: {chain_id}")
                    print(f"panoptic_pool_id: {panoptic_pool_id}")
                    raise

    return oracle_ticks_per_panoptic_pool


def get_twap_ticks_per_panoptic_pool(panoptic_pool_accounts, block, chain_id):
    TWAP_WINDOW = 600
    rpc_client = chain_to_rpc_client[chain_id]
    twap_ticks_per_panoptic_pool = {}
    panoptic_math_v1_contract = rpc_client.eth.contract(address=chain_to_contract_addrs[chain_id]['PanopticMath'], abi=PANOPTIC_MATH_ABI)
    panoptic_math_v1_1_contract  = rpc_client.eth.contract(address=chain_to_contract_addrs[chain_id]['PanopticMathV1_1'], abi=PANOPTIC_MATH_ABI)

    for ppa in panoptic_pool_accounts:
        panoptic_pool_id = ppa['panopticPool']['id']
        panoptic_pool_oracle = ppa['panopticPool']['oracleContract']
        is_v4_pool = ppa['panopticPool']['underlyingPool']['isV4Pool']
        if panoptic_pool_id not in twap_ticks_per_panoptic_pool:
            panoptic_math_contract = panoptic_math_v1_1_contract if is_v4_pool else panoptic_math_v1_contract

            if panoptic_pool_id == '0x000002daae4a265e7e0af0917cf8efcf0376ecdf':
                # Since getOracleTicks() is reverting for this mainnet pool (PRIME/USDC), use the current tick at the mainnet whitehat block - 1 instead
                # repeated 5 times to mimic the interface of getOracleTicks()
                twap_ticks_per_panoptic_pool[panoptic_pool_id] = 248277
            else:
                twap_ticks_per_panoptic_pool[panoptic_pool_id] = panoptic_math_contract.functions.twapFilter(Web3.to_checksum_address(panoptic_pool_oracle), TWAP_WINDOW).call(block_identifier=block)

    return twap_ticks_per_panoptic_pool


def calculate_user_funds_at_time_of_rescue(panoptic_pool_accounts, share_prices_per_panoptic_pool, oracle_ticks_per_panoptic_pool, block, chain_id):
    rpc_client = chain_to_rpc_client[chain_id]
    panoptic_query_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(chain_to_contract_addrs[chain_id]['PanopticQuery']), abi=PANOPTIC_QUERY_ABI)

    account_funds_per_panoptic_pool = []

    for i, ppa in enumerate(panoptic_pool_accounts):
        # Add jitter to requests
        time.sleep(random.uniform(0, 0.3))

        if i % 25 == 0:
            print(f'Processing pool account {i}')

        panoptic_pool_id = ppa['panopticPool']['id']
        panoptic_pool_address = Web3.to_checksum_address(panoptic_pool_id)
        account_id = ppa['account']['id']
        account_address = Web3.to_checksum_address(account_id)

        # TODO: delet in favor of enriching input dict
        panoptic_pool_account_funds = {
            'panoptic_pool_id': panoptic_pool_id,
            'account_id': account_id,
            'token0': { 'id': ppa['panopticPool']['token0']['id'], 'decimals': ppa['panopticPool']['token0']['decimals'], 'symbol': ppa['panopticPool']['token0']['symbol'], 'derivedETH': ppa['panopticPool']['token0']['derivedETH'] }, 
            'token1': { 'id': ppa['panopticPool']['token1']['id'], 'decimals': ppa['panopticPool']['token1']['decimals'], 'symbol': ppa['panopticPool']['token1']['symbol'], 'derivedETH': ppa['panopticPool']['token1']['derivedETH'] }, 
            'nlv0': 0,
            'nlv1': 0,
            'assets0': 0,
            'assets1': 0
        }

        collateral0_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral0']['id']), abi=COLLATERAL_TRACKER_ABI)
        collateral1_contract = rpc_client.eth.contract(address=Web3.to_checksum_address(ppa['collateral1']['id']), abi=COLLATERAL_TRACKER_ABI)

        try:
            [current_tick, fast_oracle_tick, slow_oracle_tick, latest_observation, median_data] = oracle_ticks_per_panoptic_pool[panoptic_pool_id]
        except:
            print('Error processing panoptic_pool_id:', panoptic_pool_id)
            raise
        # twap_tick = twap_ticks_per_panoptic_pool[panoptic_pool_id]

        # Get NLV if user has positions open
        position_id_list = [int(ab['tokenId']['id']) for ab in ppa['accountBalances']]
        if len(position_id_list) == 0:
            panoptic_pool_account_funds['nlv0'] = 0
            panoptic_pool_account_funds['nlv1'] = 0
            # enrich input dict
            ppa['nlv0'] = 0
            ppa['nlv1'] = 0
        else:
            # TODO: if collateral is stableocin and has no assets, can just use that stablecoin value directly
            nlv = panoptic_query_contract.functions.getNetLiquidationValue(
                panoptic_pool_address,
                account_address,
                True, # includePendingPremium
                position_id_list,
                fast_oracle_tick # atTick
                # twap_tick # atTick
            ).call(block_identifier=block)
            panoptic_pool_account_funds['nlv0'] = nlv[0]
            panoptic_pool_account_funds['nlv1'] = nlv[1]
            # enrich input dict
            ppa['nlv0'] = nlv[0]
            ppa['nlv1'] = nlv[1]

        # Fetch user's balance of shares0 and shares1 and convert to assets using share price
        share_prices = share_prices_per_panoptic_pool[panoptic_pool_id]
        shares0_balance = collateral0_contract.functions.balanceOf(account_address).call(block_identifier=block)
        assets0_balance = Decimal(shares0_balance) * share_prices[0]
        shares1_balance = collateral1_contract.functions.balanceOf(account_address).call(block_identifier=block)
        assets1_balance = Decimal(shares1_balance) * share_prices[1]
        panoptic_pool_account_funds['assets0'] = assets0_balance
        panoptic_pool_account_funds['assets1'] = assets1_balance
        # enrich input dict
        ppa['assets0'] = assets0_balance
        ppa['assets1'] = assets1_balance

        account_funds_per_panoptic_pool.append(panoptic_pool_account_funds)

    return account_funds_per_panoptic_pool
    

In [19]:
# Get accounts with deposits or open positions at block t

chain_to_eth_price_usd = {}
panoptic_pool_accounts_per_chain = {}

GetPanopticPoolAccountsWithOpenPositionsOrCollateralDepositsQuery = '''
query GetPanopticPoolAccountsWithOpenPositionsOrCollateralDeposits($blockBeforeWhitehat: Int!) {
  bundle(id:"1", block: {number: $blockBeforeWhitehat}) {
    ethPriceUSD
  }

  panopticPoolAccounts(
    block: {number: $blockBeforeWhitehat}
    first: 1000
    where: {
      and: [
        { account: "0x000a460f9e5fc39b30976cbf3484d4826941f558" }
        { or: 
          [
            {collateral0Shares_gt: 0},
            {collateral1Shares_gt: 0},
            {accountBalances_: {isOpen: 1}}
          ]
        }
      ]
    }
  ) {
    account {
      id
    }
    panopticPool {
      id
      token0 { id decimals symbol derivedETH }
      token1 { id decimals symbol derivedETH }
      oracleContract
      underlyingPool {
        isV4Pool
      }
    }
    collateral0 {
      id
    }
    collateral1 {
      id
    }
    accountBalances(
      first: 1000
      where: {isOpen: 1}
      orderBy: createdBlockNumber
      orderDirection: desc
    ) {
      tokenId {
        id
      }
    }
  }
}
'''

for chain_id in panoptic_chains:
    t = chain_to_whitehat_blocks[chain_id] - 1

    response = requests.post(
        chain_to_subgraph_url[chain_id],
        json={
            'query': GetPanopticPoolAccountsWithOpenPositionsOrCollateralDepositsQuery,
            'variables': {'blockBeforeWhitehat': t}
        }
    )

    resp = response.json()
    panoptic_pool_accounts = resp['data']['panopticPoolAccounts']
    print(f'# of ppas at block t for chain {chain_id}:', len(panoptic_pool_accounts))
    panoptic_pool_accounts_per_chain[chain_id] = panoptic_pool_accounts

    eth_price_usd = resp['data']['bundle']['ethPriceUSD']
    chain_to_eth_price_usd[chain_id] = eth_price_usd

# of ppas at block t for chain 130: 0
# of ppas at block t for chain 8453: 0
# of ppas at block t for chain 1: 1


In [20]:
# chains_to_process = panoptic_chains
chains_to_process = [1]
# chains_to_process = [130]
# chains_to_process = [8453]

In [21]:
chain_to_panoptic_pool_to_share_prices = {}
chain_to_panoptic_pool_to_oracle_ticks = {}

for chain_id in chains_to_process:
    t = chain_to_whitehat_blocks[chain_id] - 1

    # TODO: use multicall inside these funcs' rpc queries to speed them up 
    share_prices_per_panoptic_pool = get_share_prices_per_panoptic_pool(panoptic_pool_accounts_per_chain[chain_id], block=t, chain_id=chain_id)
    oracle_ticks_per_panoptic_pool = get_oracle_ticks_per_panoptic_pool(panoptic_pool_accounts_per_chain[chain_id], block=t, chain_id=chain_id)
    chain_to_panoptic_pool_to_share_prices[chain_id] = share_prices_per_panoptic_pool
    chain_to_panoptic_pool_to_oracle_ticks[chain_id] = oracle_ticks_per_panoptic_pool
    


In [22]:
share_prices_per_panoptic_pool

{'0x000000000000305b8621e2475aee38ab5721d525': [Decimal('0.000001040829395959184579736986706'),
  Decimal('0.000001031203986734423259854205578')]}

In [23]:
chain_to_panoptic_pool_to_account_funds = {}
for chain_id in chains_to_process:
    account_funds_per_panoptic_pool = calculate_user_funds_at_time_of_rescue(panoptic_pool_accounts_per_chain[chain_id], chain_to_panoptic_pool_to_share_prices[chain_id], chain_to_panoptic_pool_to_oracle_ticks[chain_id], block=t, chain_id=chain_id)
    print('account_funds_per_panoptic_pool')
    print(account_funds_per_panoptic_pool)

    chain_to_panoptic_pool_to_account_funds[chain_id] = account_funds_per_panoptic_pool

    print('Done processing {} panoptic pool accounts on chain id: {}'.format(len(panoptic_pool_accounts_per_chain[chain_id]), chain_id))

Processing pool account 0
account_funds_per_panoptic_pool
[{'panoptic_pool_id': '0x000000000000305b8621e2475aee38ab5721d525', 'account_id': '0x000a460f9e5fc39b30976cbf3484d4826941f558', 'token0': {'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'decimals': '6', 'symbol': 'USDC', 'derivedETH': '0.0002227618401010833593182198022050998'}, 'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'decimals': '18', 'symbol': 'WETH', 'derivedETH': '1'}, 'nlv0': 0, 'nlv1': 0, 'assets0': Decimal('0E-33'), 'assets1': Decimal('0.6601056392322930957537324709')}]
Done processing 1 panoptic pool accounts on chain id: 1


In [ ]:
chain_to_snapshot_df = {}
chain_id_to_weth_addr = {
    1: mainnet_weth,
    8453: base_weth,
    130: unichain_weth
}

for chain_id in chains_to_process:
    account_funds_per_panoptic_pool = chain_to_panoptic_pool_to_account_funds[chain_id]

    # Calculate net balances per (account, token) tuple
    net_balances = {}
    token_info = {} # Store token symbol and decimals

    for ppa in account_funds_per_panoptic_pool:
        account_id = ppa['account_id']
        token0_id = ppa['token0']['id']
        token0_symbol = ppa['token0']['symbol']
        token0_decimals = ppa['token0']['decimals']
        token0_derivedETH = ppa['token0']['derivedETH']
        token1_id = ppa['token1']['id']
        token1_symbol = ppa['token1']['symbol']
        token1_decimals = ppa['token1']['decimals']
        token1_derivedETH = ppa['token1']['derivedETH']
        
        # Store token info
        token_info[token0_id] = {'symbol': token0_symbol, 'decimals': token0_decimals, 'derivedETH': token0_derivedETH}
        token_info[token1_id] = {'symbol': token1_symbol, 'decimals': token1_decimals, 'derivedETH': token1_derivedETH}
        
        # Update token0 balance
        key0 = (account_id, token0_id)
        if key0 not in net_balances:
            net_balances[key0] = Decimal(0)
        net_balances[key0] += Decimal(ppa['nlv0']) + ppa['assets0']
        
        # Update token1 balance
        key1 = (account_id, token1_id)
        if key1 not in net_balances:
            net_balances[key1] = Decimal(0)
        net_balances[key1] += Decimal(ppa['nlv1']) + ppa['assets1']

    # Create DataFrame with columns:
    # account_id | token_address | token_symbol | token_decimals | balance | derivedETH
    data = []
    for (account_id, token_id), balance in net_balances.items():
        # The `balance` value is a Decimal to preserve precision when multiplying user assets by share price, which is also a Decimal
        # Convert it to an integer by dropping the fractional part
        balance_int = balance.quantize(Decimal('1'), ROUND_DOWN)
        token_symbol = token_info[token_id]['symbol']
        token_decimals = token_info[token_id]['decimals']
        token_derivedETH = token_info[token_id]['derivedETH']
        data.append([account_id, token_id, token_symbol, token_decimals, str(balance_int), token_derivedETH])

    df = pd.DataFrame(data, columns=['account_id', 'token_address', 'token_symbol', 'token_decimals', 'balance', 'derivedETH'])

    # TODO: instead of manipulating ETH -> WETH, sum the balances together

    # Coerce rows with ETH symbol and address -> WETH since the attacker contract wrapped native ETH from v4 pools
    eth_rows = df['token_symbol'] == 'ETH'
    df.loc[eth_rows, ['token_symbol', 'token_address']] = ['WETH', chain_id_to_weth_addr[chain_id].lower()]

    df.to_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.csv', index=False)
    chain_to_snapshot_df[chain_id] = df

## Calculate USD value of user balances on each chain

In [ ]:
eth_price_usd = Decimal(chain_to_eth_price_usd[1])

# given a df with column `balance` in raw token amounts and col token_decimals, use Decimal to return full precision formatted balances
def format_balance(row):
    balance = Decimal(str(row['balance']))
    decimals = Decimal(str(row['token_decimals']))
    divisor = Decimal(10) ** decimals
    return balance / divisor

for chain_id in panoptic_chains:
    snapshot_df = pd.read_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.csv')
    
    # Calculate USD value of tokens
    # convert everything to Decimal for max precision
    balances_formatted = snapshot_df.apply(format_balance, axis=1)
    derived_eth = snapshot_df['derivedETH'].apply(lambda x: Decimal(str(x)))
    snapshot_df.loc[:, 'usd_value'] = balances_formatted * derived_eth * eth_price_usd
    
    # Export modified dataframe back to CSV
    snapshot_df.to_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances_w_usd.csv', index=False)
    
    print('Total USD value for chain: ', chain_id_to_chain_name[chain_id])
    total_usd_value = snapshot_df['usd_value'].sum()
    print(total_usd_value)



Total USD value for chain:  unichain
35153.86252273613831395140636
Total USD value for chain:  base
97592.61780847123525343179750
Total USD value for chain:  ethereum
431281.1616303159110928750240


## Find how many additional tokens of the kind in our Safe we need to acquire to fully repay users in-kind

## Tokens in safe

In [344]:
# for chain_id in [1]:
for chain_id in panoptic_chains:
    print("Calculating shortfall for chain_id", chain_id)
    path = chain_id_to_chain_name[chain_id]
    
    safe_df = pd.read_csv(f'./{chain_id_to_chain_name[chain_id]}_funds.csv', dtype={'token_address': str, 'balance': str})
    balances_df = pd.read_csv(f'./balances/{chain_id_to_chain_name[chain_id]}_balances.csv', dtype={'token_address': str, 'balance': str})
    # convert to Python native int, not pandas default int representation, numpys int64, because int64 is not precise enough. got overflow errors parsing MOG balance for example
    balances_df['balance'] = balances_df['balance'].apply(int)
    safe_df['balance'] = safe_df['balance'].apply(int)

    # Get set of token addresses from safe
    safe_token_addrs = set(safe_df['token_address'])
    print(f"safe_token_addrs: ${safe_token_addrs}")
    
    # Filter balances to only include tokens in safe
    filtered_df = balances_df[balances_df['token_address'].isin(safe_df['token_address'])]
    
    print(f"Found {len(filtered_df)} rows with tokens that match the safe")
    # Sum balances by token address
    token_sums = filtered_df.groupby('token_address')['balance'].sum()
    token_sums.to_csv(f'balances/{chain_id_to_chain_name[chain_id]}_token_sums.csv', index=True)
    
    # Calculate shortfall by subtracting safe balances from user balances
    shortfalls = {}
    for token_addr in safe_token_addrs:
        safe_balance = safe_df[safe_df['token_address'] == token_addr]['balance'].iloc[0]
        user_balance = token_sums.get(token_addr, 0)  # Default to 0 if token not in user balances
        shortfall = user_balance - safe_balance
        print('token-addr', token_addr)
        print('user_balance', user_balance)
        print('safe_balance', safe_balance)
        shortfalls[token_addr] = shortfall
        
    print("\nToken shortfalls (positive means need to acquire more):")
    for token_addr, shortfall in shortfalls.items():
        token_symbol = safe_df[safe_df['token_address'] == token_addr]['token_symbol'].iloc[0]
        print(f"{token_symbol}: {shortfall}")
    print("\n\n***\n\n")


Calculating shortfall for chain_id 130
safe_token_addrs: ${'0x078d782b760474a361dda0af3839290b0ef57ad6', '0x9151434b16b9763660705744891fa906f660ecc5', '0x4200000000000000000000000000000000000006', '0x927b51f251480a681271180da4de28d44ec4afb8'}
Found 760 rows with tokens that match the safe
token-addr 0x078d782b760474a361dda0af3839290b0ef57ad6
user_balance 8953705377
safe_balance 8975993158
token-addr 0x9151434b16b9763660705744891fa906f660ecc5
user_balance 76106964
safe_balance 12979404
token-addr 0x4200000000000000000000000000000000000006
user_balance 5454920983002269807
safe_balance 5267119200645429045
token-addr 0x927b51f251480a681271180da4de28d44ec4afb8
user_balance 1492455
safe_balance 1488291

Token shortfalls (positive means need to acquire more):
USDC: -22287781
USDT: 63127560
WETH: 187801782356840762
WBTC: 4164


***


Calculating shortfall for chain_id 8453
safe_token_addrs: ${'0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf', '0x4200000000000000000000000000000000000006', '0x833589f

## Tokens *NOT* in safe

In [5]:
# for chain_id in [1]:
for chain_id in panoptic_chains:
    print("Calculating shortfall for chain_id", chain_id)
    path = chain_id_to_chain_name[chain_id]
    
    safe_df = pd.read_csv(f'./{chain_id_to_chain_name[chain_id]}_funds.csv', dtype={'token_address': str, 'balance': str})
    balances_df = pd.read_csv(f'./balances/{chain_id_to_chain_name[chain_id]}_balances.csv', dtype={'token_address': str, 'balance': str})
    # convert to Python native int, not pandas default int representation, numpys int64, because int64 is not precise enough. got overflow errors parsing MOG balance for example
    balances_df['balance'] = balances_df['balance'].apply(int)
    safe_df['balance'] = safe_df['balance'].apply(int)

    # Get set of token addresses from safe
    safe_token_addrs = set(safe_df['token_address'])
    print(f"safe_token_addrs: ${safe_token_addrs}")
    
    # Filter balances to only include tokens NOT in safe
    filtered_df = balances_df[~balances_df['token_address'].isin(safe_df['token_address'])]
    
    print(f"Found {len(filtered_df)} rows with tokens that do not match the safe")
    # Sum balances by token address
    token_sums = filtered_df.groupby('token_address')['balance'].sum()
    token_sums.to_csv(f'balances/{chain_id_to_chain_name[chain_id]}_token_sums_not_in_safe.csv', index=True)
    
    # Calculate shortfall by subtracting safe balances from user balances
    shortfalls = {}
    for token_addr in set(filtered_df['token_address']):
        user_balance = token_sums.get(token_addr, 0)  # Default to 0 if token not in user balances
        shortfalls[token_addr] = user_balance
        
    print("\nToken sums for tokens not in safe:")
    for token_addr, sum_balance in shortfalls.items():
        print(f"Token Address: {token_addr}, Sum Balance: {sum_balance}")

    print("\n\n***\n\n")


Calculating shortfall for chain_id 130
safe_token_addrs: ${'0x078d782b760474a361dda0af3839290b0ef57ad6', '0x4200000000000000000000000000000000000006', '0x9151434b16b9763660705744891fa906f660ecc5', '0x927b51f251480a681271180da4de28d44ec4afb8'}
Found 49 rows with tokens that do not match the safe

Token sums for tokens not in safe:
Token Address: 0x8f187aa05619a017077f5308904739877ce9ea21, Sum Balance: 2204998233159092432
Token Address: 0x20cab320a855b39f724131c69424240519573f81, Sum Balance: 182192442248577181


***


Calculating shortfall for chain_id 8453
safe_token_addrs: ${'0x4200000000000000000000000000000000000006', '0x833589fcd6edb6e08f4c7c32d4f71b54bda02913', '0xcbb7c0000ab88b473b1f5afd9ef808440eed33bf'}
Found 7 rows with tokens that do not match the safe

Token sums for tokens not in safe:
Token Address: 0x940181a94a35a4569e4529a3cdfb74e38fd98631, Sum Balance: 1000000430328497449
Token Address: 0x2da56acb9ea78330f947bd57c54119debda7af71, Sum Balance: 199414341994528776900726934

In [1]:
# mainnet
# Finally, as a sanity check, we net all user balances for each token - this should be within a 6% percent of the respective token's balance held in our safe.
user_weth = 0
user_wbtc = 0
user_tbtc = 0
user_usdc = 0

for (account_id, token_id), balance in net_balances.items():
    balance_int = balance.quantize(Decimal('1'), ROUND_DOWN)
    if token_id == "0x0000000000000000000000000000000000000000" or token_id.lower() == mainnet_weth.lower():
        user_weth += balance_int
    elif token_id.lower() == mainnet_wbtc.lower():
        user_wbtc += balance_int
    elif token_id.lower() == mainnet_tbtc.lower():
        user_tbtc += balance_int
    elif token_id.lower() == mainnet_usdc.lower():
        user_usdc += balance_int

# Convert safe balances to Decimal for comparison
safe_weth = weth_balance
safe_wbtc = wbtc_balance
safe_tbtc = tbtc_balance
safe_usdc = usdc_balance

# Calculate percentage differences
def calc_percentage_diff(actual, expected):
    if expected == 0:
        return float('inf') if actual != 0 else 0
    return abs(actual - expected) / expected * 100

weth_diff = calc_percentage_diff(user_weth, safe_weth)
wbtc_diff = calc_percentage_diff(user_wbtc, safe_wbtc)
tbtc_diff = calc_percentage_diff(user_tbtc, safe_tbtc)
usdc_diff = calc_percentage_diff(user_usdc, safe_usdc)

print("Net balances vs Safe balances comparison:")
print(f"WETH: {user_weth} vs {safe_weth} (diff: {weth_diff:.2f}%)")
print(f"WBTC: {user_wbtc} vs {safe_wbtc} (diff: {wbtc_diff:.2f}%)")
print(f"TBTC: {user_tbtc} vs {safe_tbtc} (diff: {tbtc_diff:.2f}%)")
print(f"USDC: {user_usdc} vs {safe_usdc} (diff: {usdc_diff:.2f}%)")

# Check if any differences are above 6%
threshold = 6
print("\nValidation results:")
if weth_diff > threshold:
    print(f"WARNING: WETH difference ({weth_diff:.2f}%) exceeds {threshold}% threshold")
if wbtc_diff > threshold:
    print(f"WARNING: WBTC difference ({wbtc_diff:.2f}%) exceeds {threshold}% threshold")
if tbtc_diff > threshold:
    print(f"WARNING: TBTC difference ({tbtc_diff:.2f}%) exceeds {threshold}% threshold")
if usdc_diff > threshold:
    print(f"WARNING: USDC difference ({usdc_diff:.2f}%) exceeds {threshold}% threshold")

NameError: name 'net_balances' is not defined

In [ ]:
for chain_id in panoptic_chains:




## TEMP

In [34]:
# Collapse duplicate WETH rows to one by summing balances

for chain_id in panoptic_chains:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.bk.og.csv', dtype={
        'derivedETH': str,
        'balance': object # to use Int  under the hood instead of string for really big values or numpy int64 for smaller values
    })

    # Convert the 'balance' column values to Python int types which has inf precision, unlike
    df['balance'] = df['balance'].apply(lambda x: int(x))

    # Group by 'account_id' and 'token_address', summing the 'balance' values
    collapsed_df = df.groupby(['account_id', 'token_address'], as_index=False).agg({
        'token_symbol': 'first',
        'token_decimals': 'first',
        'balance': 'sum',
        'derivedETH': 'first'
    })

    # Save the collapsed DataFrame back to a CSV file
    collapsed_df.to_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.csv', index=False)


In [ ]:
# Add formatted balance to clear up user confusion

def formatUnits(value, decimals):
    display = str(value)
    
    negative = display.startswith('-')
    if negative:
        display = display[1:]
        
    display = display.zfill(decimals)
    
    integer = display[:-decimals] if len(display) > decimals else '0'
    fraction = display[-decimals:].rstrip('0')
    
    return f"{'-' if negative else ''}{integer}{'.' + fraction if fraction else ''}"

for chain_id in panoptic_chains:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.csv',  dtype={
        # Read as strings to preserve precision
        'derivedETH': str, 
        'balance': str
    })


for chain_id in panoptic_chains:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.csv',  dtype={
        # Read as string to preserve precision
        'derivedETH': str,
        'balance': str
    })

    # Add formattedBalances column by applying formatUnits function to each row
    df['formattedBalances'] = df.apply(lambda row: formatUnits(row['balance'], row['token_decimals']), axis=1)

    # Save the DataFrame with formattedBalances to a new CSV file
    df.to_csv(f'balances/{chain_id_to_chain_name[chain_id]}_balances.csv', index=False)